# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [3]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [4]:
questions, answers = get_questions_and_answers(faq_path="../data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [5]:
from sentence_transformers import SentenceTransformer, util
import spacy


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(message)

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def remove_stop_words(text:str):
    nlp_model = spacy.load("pt_core_news_sm")
    text_tokens = nlp_model(text)

    words_without_stopwords = []

    for token in text_tokens:
        if not token.is_stop:
            words_without_stopwords.append(token.text)
    
    text_without_stopwords = " ".join(words_without_stopwords)

    return text_without_stopwords

In [6]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [7]:
message = "Eae, beleza? asdasd gostaria de saber se é possível melhorar meu plano?"
message = remove_stop_words(message)
print(message)

Eae , beleza ? asdasd gostaria melhorar plano ?


### 2.1.1 all-mpnet-base-v2 model

In [8]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...


### 2.1.2 multi-qa-mpnet-base-dot-v1

In [12]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
12,O que fazer em caso de falha no serviço de int...,"Em caso de falha no serviço, entre em contato ..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...


### 2.1.3 all-distilroberta-v1 model

In [13]:
model = SentenceTransformer("all-distilroberta-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
20,Quais são os benefícios de ser um cliente de l...,Oferecemos programas de fidelidade e benefício...
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."


### 2.1.4 all-MiniLM-L6-v2 model

In [14]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
14,A empresa oferece roteadores e equipamentos pa...,"Sim, fornecemos equipamentos para internet em ..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."


### 2.1.5 multi-qa-MiniLM-L6-cos-v1

In [15]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
25,Quais são as opções de pacotes de TV por assin...,Temos diferentes pacotes de TV por assinatura ...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."


### 2.1.6 msmarco-distilbert-base-v4

In [16]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

Eae , beleza ? asdasd gostaria melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
12,O que fazer em caso de falha no serviço de int...,"Em caso de falha no serviço, entre em contato ..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."


# 3 Preparing to Deploy

In [22]:
embeddings_model = SentenceTransformer("msmarco-distilbert-base-v4")
nlp_model = spacy.load("pt_core_news_sm")

In [23]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


## 3.1 Preparing Data

### 3.1.1 Creating Embeddings for each question and exporting as CSV file

In [24]:
df_embeddings = get_sentence_embeddings(model, df.Question)
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.001516,-0.016571,-0.016486,-0.008458,0.010217,-0.030883,-0.031925,0.007572,-0.024302,0.014526,...,-0.021148,-0.000027,-0.037820,0.022573,-0.023144,-0.038001,0.001784,0.016931,-0.010248,-0.003186
1,-0.048698,-0.058732,-0.049140,-0.050720,0.019374,-0.031220,-0.003395,-0.087827,-0.029829,-0.066374,...,0.021898,-0.054348,-0.062176,-0.049850,-0.050851,-0.013530,-0.059068,-0.059299,-0.016155,-0.022709
2,-0.008875,-0.008175,-0.020079,-0.024734,-0.018126,-0.030824,0.005096,-0.005319,0.002793,-0.044988,...,-0.033110,-0.022896,-0.021107,-0.019664,-0.019609,-0.000211,-0.013587,-0.005483,-0.025578,-0.027519
3,-0.056031,-0.033219,-0.025261,-0.018618,-0.013365,-0.035369,-0.005387,-0.014066,-0.036233,0.022442,...,0.003666,-0.046419,-0.041420,-0.033946,0.013572,-0.002355,-0.059162,0.006135,-0.011673,-0.033107
4,0.043385,0.023201,0.000391,0.037526,-0.026126,0.043609,-0.022499,0.064638,0.059636,0.083580,...,0.036733,0.035959,0.033926,0.045156,0.011013,0.025360,0.029124,0.022294,0.030505,0.015260


In [25]:
df_embeddings.to_csv(path_or_buf="../data/processed/Embeddings.csv", sep=";", index=False)

## 3.2 Exporting the model with the best performance

### 3.2.1 Embeddings model

In [26]:
embeddings_model.save('../models/embed_model')

### 3.2.2 NLP Model

In [27]:
nlp_model.to_disk('../models/nlp_model')